In [1]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from pylab import rcParams
from scipy.ndimage.interpolation import rotate, zoom
import scipy as sp
%matplotlib inline

In [ ]:
imgPaths = glob.glob("../imgs/whale_*/w_*") # not test

nObs = len(imgPaths)
subset = np.random.randint(0,nObs,10)

In [ ]:
for i in range(nObs)[:]:
    imgPath = imgPaths[i]
    x,mask = Image.open(imgPath), Image.open(imgPath.replace("w_","lf_"))
    w, h = x.size
    
    
    mask = mask.resize((w,h),Image.ANTIALIAS)
    x = np.array(x)
    threshold = 10
    maskThr = np.array(mask.point(lambda p: p > threshold and 255))
    maskThr = np.tile(np.expand_dims(maskThr.astype(bool),3),(1,1,3))
    
    try:
        mx, my, _ = map(int,sp.ndimage.center_of_mass(maskThr))

        w2 = 300
        x1 = mx-w2
        y2 = my+w2
        y1 = my-w2
        x2 = mx+w2

        cropped = x.copy()
        cropped = cropped[x1:x2,y1:y2]
        cropped = Image.fromarray(cropped)
        cropped.save(imgPath.replace("w_","hS_"))
        def showCropped():
            plt.imshow(cropped)
            plt.title(imgPath)
            plt.show()
            
        if np.random.uniform()< 0.01:
            showCropped()
    except ValueError:
        plt.title(imgPath + " (COULDN'T CROP)")
        plt.imshow(np.hstack((x,maskThr)))
        plt.show()